In [1]:
import pyrosm
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# load openstreetmap basemap
# https://www.openstreetmap.org/export - boundingbox setup
bbox = [16.3837, 49.0946, 16.9227, 49.2884]  # [minx, miny, maxx, maxy]
pbf_reader = pyrosm.OSM('datasets/czech_republic-latest.osm.pbf', bounding_box=bbox)
cycling_network = pbf_reader.get_network('cycling')
driving_network = pbf_reader.get_network('driving')
basemap_network = driving_network.append(cycling_network)

# load strava data
strava_df = pd.read_csv('datasets/strava_edges_2021.csv')

# load bike-to-work data
biketowork_df = gpd.read_file('datasets/do_prace_na_kole.geojson')

# load mechanical counters data
counters_df = gpd.read_file('datasets/cyklodetektory.geojson')

# load driving census data
census_df = pd.read_excel('datasets/rsd_scitanie_dopravy_2020.xlsx', skiprows=[0])

/tmp/ipykernel_205321/1491798065.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  basemap_network = driving_network.append(cycling_network)


In [3]:
print(cycling_network.shape)
print(driving_network.shape)
print(basemap_network.shape)
print(strava_df.shape)

(36644, 39)
(23179, 39)
(59823, 40)
(193812, 34)


In [24]:
# group edges into one OSM way (some columns will need to be averaged, not summed)
strava_grouped_df = strava_df.groupby(['osm_reference_id']).sum()
model = strava_grouped_df.join(basemap_network.set_index('id'), on='osm_reference_id', how='inner')
model = model.drop_duplicates()
# https://www.openstreetmap.org/way/{id}
model.head()

,edge_uid,year,forward_trip_count,reverse_trip_count,forward_people_count,reverse_people_count,forward_commute_trip_count,reverse_commute_trip_count,forward_leisure_trip_count,reverse_leisure_trip_count,...,tunnel,turn,width,timestamp,version,tags,osm_type,geometry,length,path
osm_reference_id,,,,,,,,,,,,,,,,,,,,,
4019269,161052016,2021,30,0,10,0,0,0,30,0,...,None,None,None,1556088822,16,"{""layer"":""1"",""source:maxspeed"":""CZ:urban_trunk...",way,"MULTILINESTRING ((16.56569 49.18349, 16.56527 ...",81.0,NaN
4019432,161020360,2021,805,745,265,270,140,105,665,640,...,None,None,None,1611510572,10,"{""source:maxspeed"":""CZ:rural"",""source:ref"":""rs...",way,"MULTILINESTRING ((16.52347 49.17080, 16.52324 ...",159.0,NaN
4019434,161039603,2021,5,0,5,0,0,0,5,0,...,None,None,None,1530611385,13,"{""source:maxspeed"":""sign""}",way,"MULTILINESTRING ((16.54352 49.16684, 16.54319 ...",380.0,NaN
4019435,322085221,4042,50,0,25,0,20,0,30,0,...,None,None,None,1639339626,10,"{""source"":""uhul:ortofoto""}",way,"MULTILINESTRING ((16.56420 49.17379, 16.56420 ...",338.0,NaN
4019437,644174524,8084,1100,990,140,220,55,55,1045,935,...,None,None,None,1648299982,16,None,way,"MULTILINESTRING ((16.56755 49.16300, 16.56762 ...",166.0,NaN


In [26]:
print(model.shape)
model

(27050, 71)


,edge_uid,year,forward_trip_count,reverse_trip_count,forward_people_count,reverse_people_count,forward_commute_trip_count,reverse_commute_trip_count,forward_leisure_trip_count,reverse_leisure_trip_count,...,tunnel,turn,width,timestamp,version,tags,osm_type,geometry,length,path
osm_reference_id,,,,,,,,,,,,,,,,,,,,,
4019269,161052016,2021,30,0,10,0,0,0,30,0,...,None,None,None,1556088822,16,"{""layer"":""1"",""source:maxspeed"":""CZ:urban_trunk...",way,"MULTILINESTRING ((16.56569 49.18349, 16.56527 ...",81.0,NaN
4019432,161020360,2021,805,745,265,270,140,105,665,640,...,None,None,None,1611510572,10,"{""source:maxspeed"":""CZ:rural"",""source:ref"":""rs...",way,"MULTILINESTRING ((16.52347 49.17080, 16.52324 ...",159.0,NaN
4019434,161039603,2021,5,0,5,0,0,0,5,0,...,None,None,None,1530611385,13,"{""source:maxspeed"":""sign""}",way,"MULTILINESTRING ((16.54352 49.16684, 16.54319 ...",380.0,NaN
4019435,322085221,4042,50,0,25,0,20,0,30,0,...,None,None,None,1639339626,10,"{""source"":""uhul:ortofoto""}",way,"MULTILINESTRING ((16.56420 49.17379, 16.56420 ...",338.0,NaN
4019437,644174524,8084,1100,990,140,220,55,55,1045,935,...,None,None,None,1648299982,16,None,way,"MULTILINESTRING ((16.56755 49.16300, 16.56762 ...",166.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023721767,161938969,2021,5,0,0,0,0,0,5,0,...,None,None,None,1644149494,2,None,way,"MULTILINESTRING ((16.58372 49.24982, 16.58388 ...",27.0,NaN
1023721771,161930395,2021,170,395,105,170,10,15,160,380,...,None,None,None,1642884868,1,None,way,"MULTILINESTRING ((16.56073 49.25148, 16.56098 ...",363.0,NaN
1023721772,161939674,2021,40,5,35,5,15,0,25,5,...,None,None,None,1661716762,2,None,way,"MULTILINESTRING ((16.58314 49.25817, 16.58319 ...",111.0,NaN


In [23]:
model.to_csv('model.csv', sep=';')